# Manage memory

Many AI applications need memory to share context across multiple interactions. LangGraph supports two types of memory essential for building conversational agents:

- [Short-term memory](#add-short-term-memory): Tracks the ongoing conversation by maintaining message history within a session.
- [Long-term memory](#add-long-term-memory): Stores user-specific or application-level data across sessions.

With [short-term memory](#add-short-term-memory) enabled, long conversations can exceed the LLM's context window. Common solutions are:

* [Trimming](#trim-messages): Remove first or last N messages (before calling LLM)
* [Summarization](#summarize-messages): Summarize earlier messages in the history and replace them with a summary
* [Delete messages](#delete-messages) from LangGraph state permanently
* custom strategies (e.g., message filtering, etc.)

This allows the agent to keep track of the conversation without exceeding the LLM's context window.

In [ ]:
# hide-cell
%%capture --no-stderr
%pip install --quiet -U langgraph "langchain[anthropic]"

In [ ]:
# hide-cell
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

## Add short-term memory

Short-term memory enables agents to track multi-turn conversations:

```python
# highlight-next-line
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph

# highlight-next-line
checkpointer = InMemorySaver()

builder = StateGraph(...)
# highlight-next-line
graph = builder.compile(checkpointer=checkpointer)

graph.invoke(
    {"messages": [{"role": "user", "content": "hi! i am Bob"}]},
    # highlight-next-line
    {"configurable": {"thread_id": "1"}},
)
```

See the [persistence](../persistence#add-short-term-memory) guide to learn more about working with short-term memory.

## Add long-term memory

Use long-term memory to store user-specific or application-specific data across conversations. This is useful for applications like chatbots, where you want to remember user preferences or other information.

```python
# highlight-next-line
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph

# highlight-next-line
store = InMemoryStore()

builder = StateGraph(...)
# highlight-next-line
graph = builder.compile(store=store)
```

See the [persistence](../persistence#add-long-term-memory) guide to learn more about working with long-term memory.

## Process model input

When managing long conversation history, you will need to preprocess the list of messages before sending it to the LLM (e.g., [trim](#trim-messages), [summarize](#summarize-messages) or [delete](#delete-messages) messages).

### Handle cheap computation

If the preprocessing is cheap and doesn't need to be persisted or streamed separately, you can add it directly to the model-calling graph node:

```python
def process_messages(messages: list[AnyMessage]):
    # keep only the last message
    return messages[-1:]

def call_model(state: MessagesState):
    # highlight-next-line
    messages = process_messages(state["messages"])
    response = model.invoke(messages)
    return {"messages": [response]}


builder = StateGraph(MessagesState)
builder.add_node(call_model)
...
```

??? example "Full example: process messages in a model-calling node"

    ```python
    from langchain.chat_models import init_chat_model
    from langchain_core.messages import AnyMessage
    
    from langgraph.graph import MessagesState, StateGraph, START
    from langgraph.checkpoint.memory import InMemorySaver
    
    def process_messages(messages: list[AnyMessage]):
        # keep only the last message
        return messages[-1:]
    
    def call_model(state: MessagesState):
        # highlight-next-line
        messages = process_messages(state["messages"])
        response = model.invoke(messages)
        return {"messages": [response]}
    
    
    # highight-next-line
    checkpointer = InMemorySaver()
    model = init_chat_model("anthropic:claude-3-5-haiku-latest")
    
    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")
    # highight-next-line
    app = builder.compile(checkpointer=checkpointer)
    
    config = {"configurable": {"thread_id": "1"}}
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="values"
    ):
        event["messages"][-1].pretty_print()
    
    # This will now not remember the previous messages
    # because we set `messages[-1:]` in the process_messages function
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="values"
    ):
        event["messages"][-1].pretty_print()
    ```

    ```
    ================================ Human Message =================================
    
    hi! I'm bob
    ================================== Ai Message ==================================
    
    Hi Bob! How are you doing today? Is there anything I can help you with?
    ================================ Human Message =================================
    
    what's my name?
    ================================== Ai Message ==================================
    
    I apologize, but you haven't told me your name yet. Could you please introduce yourself?
    ```

### Handle expensive computation

If the preprocessing step involves **expensive computation** (e.g., message [summarization](#summarize-messages)) and needs to be persisted or streamed separately, add the input processing as a separate graph node:

```python
from typing_extensions import TypedDict

class OverallState(MessagesState):
    some_key: str

class LLMInputState(TypedDict):
    # highlight-next-line
    llm_input_messages: list[AnyMessage]

def process_messages(state: MessagesState):
    return {
        # keep only the last message
        "llm_input_messages": state["messages"][-1:],  # (1)!
        "some_key": "some_value"  # (2)!
    }

# highlight-next-line
def call_model(state: LLMInputState):  # (3)!
    response = model.invoke(state["llm_input_messages"])
    return {"messages": response}

builder = StateGraph(OverallState)
# highlight-next-line
builder.add_node(process_messages)
builder.add_node(call_model)
...
```

1. `llm_input_messages` will be visible only to the `call_model` node and won't be added to overall graph state.
2. `some_key` will be added to the **overall** graph state.
3. `call_model` uses a private input schema, different from the `OverallState`. This allows us to change the input to `call_model` node without affecting the **overall** graph state.

??? example "Full example: process messages in a separate node"

    ```python
    from typing_extensions import TypedDict
    
    class OverallState(MessagesState):
        some_key: str
    
    
    class LLMInputState(TypedDict):
        # highlight-next-line
        llm_input_messages: list[AnyMessage]
    
    
    def process_messages(state: MessagesState):
        return {
            # keep only the last message
            "llm_input_messages": state["messages"][-1:],  # (1)!
            "some_key": "some_value"  # (2)!
        }
    
    
    # highlight-next-line
    def call_model(state: LLMInputState):  # (3)!
        response = model.invoke(state["llm_input_messages"])
        return {"messages": response}
    
    
    checkpointer = InMemorySaver()
    model = init_chat_model("anthropic:claude-3-5-haiku-latest")
    
    builder = StateGraph(OverallState)
    # highlight-next-line
    builder.add_node(process_messages)
    builder.add_node(call_model)
    builder.add_edge(START, "process_messages")
    builder.add_edge("process_messages", "call_model")
    
    app = builder.compile(checkpointer=checkpointer)
    
    config = {"configurable": {"thread_id": "1"}}
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="updates"
    ):
        print(event)
        print("\n")
    
    # This will now not remember the previous messages
    # because we set `messages[-1:]` in the filter_messages function
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="updates"
    ):
        print(event)
        print("\n")
    ```
    
    1. `llm_input_messages` will be visible only to the `call_model` node and won't be added to overall graph state.
    2. `some_key` will be added to the **overall** graph state.
    3. `call_model` uses a private input schema, different from the `OverallState`. This allows us to change the input to `call_model` node without affecting the **overall** graph state.

    ```
    {'process_messages': {'llm_input_messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='6eb8c38a-7f7c-4c7e-98ab-2e01e3e860a9')], 'some_key': 'some_value'}}


    {'call_model': {'messages': AIMessage(content='Hi Bob! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={'id': 'msg_01SoZtnvMLsSJCQdL6ajXFK8', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 21}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-d8998b94-4f49-494b-9b91-85f676cba22a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})}}
    
    
    {'process_messages': {'llm_input_messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='9a38c63e-f23d-4c38-a06d-7d7c74ade462')], 'some_key': 'some_value'}}
    
    
    {'call_model': {'messages': AIMessage(content="I apologize, but I don't know your name. I can't automatically know personal details about you unless you've previously told me. Could you introduce yourself?", additional_kwargs={}, response_metadata={'id': 'msg_01EZbuZrc65PKy8gGF4TcB3i', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 36}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-b375691b-078a-4a87-a642-8bf72a96c198-0', usage_metadata={'input_tokens': 12, 'output_tokens': 36, 'total_tokens': 48, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})}}
    ```

## Trim messages

TODO: populate this

## Summarize messages

An effective strategy for handling long conversation history is to summarize earlier messages once they reach a certain threshold:

```python
from typing import Any, TypedDict

from langchain_core.messages import AnyMessage
from langchain_core.messages.utils import count_tokens_approximately
# highlight-next-line
from langmem.short_term import SummarizationNode
from langgraph.graph import StateGraph, START, MessagesState

class State(MessagesState):
    # highlight-next-line
    context: dict[str, Any]  # (1)!

class LLMInputState(TypedDict):  # (2)!
    summarized_messages: list[AnyMessage]
    context: dict[str, Any]

# highlight-next-line
summarization_node = SummarizationNode(
    token_counter=count_tokens_approximately,
    model=summarization_model,
    max_tokens=512,
    max_tokens_before_summary=256,
    max_summary_tokens=256,
)

# highlight-next-line
def call_model(state: LLMInputState):  # (3)!
    response = model.invoke(state["summarized_messages"])
    return {"messages": [response]}

builder = StateGraph(State)
builder.add_node(call_model)
# highlight-next-line
builder.add_node("summarize", summarization_node)
builder.add_edge(START, "summarize")
builder.add_edge("summarize", "call_model")
...
```

1. We will keep track of our running summary in the `context` field
(expected by the `SummarizationNode`).
2. Define private state that will be used only for filtering
the inputs to `call_model` node.
3. We're passing a private input state here to isolate the messages returned by the summarization node

??? example "Full example: summarize messages"

    ```python
    from typing import Any, TypedDict
    
    from langchain.chat_models import init_chat_model
    from langchain_core.messages import AnyMessage
    from langchain_core.messages.utils import count_tokens_approximately
    from langgraph.graph import StateGraph, START, MessagesState
    from langgraph.checkpoint.memory import InMemorySaver
    # highlight-next-line
    from langmem.short_term import SummarizationNode
    
    model = init_chat_model("anthropic:claude-3-7-sonnet-latest")
    summarization_model = model.bind(max_tokens=128)
    
    class State(MessagesState):
        # highlight-next-line
        context: dict[str, Any]  # (1)!
    
    class LLMInputState(TypedDict):  # (2)!
        summarized_messages: list[AnyMessage]
        context: dict[str, Any]
    
    # highlight-next-line
    summarization_node = SummarizationNode(
        token_counter=count_tokens_approximately,
        model=summarization_model,
        max_tokens=256,
        max_tokens_before_summary=256,
        max_summary_tokens=128,
    )

    # highlight-next-line
    def call_model(state: LLMInputState):  # (3)!
        response = model.invoke(state["summarized_messages"])
        return {"messages": [response]}
    
    checkpointer = InMemorySaver()
    builder = StateGraph(State)
    builder.add_node(call_model)
    # highlight-next-line
    builder.add_node("summarize", summarization_node)
    builder.add_edge(START, "summarize")
    builder.add_edge("summarize", "call_model")
    graph = builder.compile(checkpointer=checkpointer)
    
    # Invoke the graph
    config = {"configurable": {"thread_id": "1"}}
    graph.invoke({"messages": "hi, my name is bob"}, config)
    graph.invoke({"messages": "write a short poem about cats"}, config)
    graph.invoke({"messages": "now do the same but for dogs"}, config)
    final_response = graph.invoke({"messages": "what's my name?"}, config)

    final_response["messages"][-1].pretty_print()
    print("\nSummary:", final_response["context"]["running_summary"].summary)
    ```

    1. We will keep track of our running summary in the `context` field
    (expected by the `SummarizationNode`).
    2. Define private state that will be used only for filtering
    the inputs to `call_model` node.
    3. We're passing a private input state here to isolate the messages returned by the summarization node

    ```
    ================================== Ai Message ==================================

    From our conversation, I can see that you introduced yourself as Bob. That's the name you shared with me when we began talking.
    
    Summary: In this conversation, I was introduced to Bob, who then asked me to write a poem about cats. I composed a poem titled "The Mystery of Cats" that captured cats' graceful movements, independent nature, and their special relationship with humans. Bob then requested a similar poem about dogs, so I wrote "The Joy of Dogs," which highlighted dogs' loyalty, enthusiasm, and loving companionship. Both poems were written in a similar style but emphasized the distinct characteristics that make each pet special.
    ```

## Delete messages

To delete messages from the graph state, you can use the `RemoveMessage`.

* Remove specific messages:

    ```python
    # highlight-next-line
    from langchain_core.messages import RemoveMessage
    
    def delete_messages(state):
        messages = state["messages"]
        if len(messages) > 2:
            # remove the earliest two messages
            # highlight-next-line
            return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}
    ```

* Remove **all** messages:
    
    ```python
    # highlight-next-line
    from langgraph.graph.message import REMOVE_ALL_MESSAGES
    
    def delete_messages(state):
        # highlight-next-line
        return {"messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)]}
    ```

!!! important "`add_messages` reducer"

    For `RemoveMessage` to work, you need to use a state key with [`add_messages`][langgraph.graph.message.add_messages] [reducer](../../../concepts/low_level#reducers), like [`MessagesState`](../../../concepts/low_level#messagesstate)

!!! warning "Valid message history"

    When deleting messages, **make sure** that the resulting message history is valid. Check the limitations of the LLM provider you're using. For example:
    
    * some providers expect message history to start with a `user` message
    * most providers require `assistant` messages with tool calls to be followed by corresponding `tool` result messages.

??? example "Full example: delete messages"

    ```python
    # highlight-next-line
    from langchain_core.messages import RemoveMessage
    
    def delete_messages(state):
        messages = state["messages"]
        if len(messages) > 2:
            # remove the earliest two messages
            # highlight-next-line
            return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}
    
    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}
    
    builder = StateGraph(MessagesState)
    builder.add_sequence([call_model, delete_messages])
    builder.add_edge(START, "call_model")
    
    checkpointer = InMemorySaver()
    app = builder.compile(checkpointer=checkpointer)
    
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="values"
    ):
        print([(message.type, message.content) for message in event["messages"]])
    
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="values"
    ):
        print([(message.type, message.content) for message in event["messages"]])
    ```

    ```
    [('human', "hi! I'm bob")]
    [('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?')]
    [('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?'), ('human', "what's my name?")]
    [('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?'), ('human', "what's my name?"), ('ai', 'Your name is Bob.')]
    [('human', "what's my name?"), ('ai', 'Your name is Bob.')]
    ```